Exercise 1 - Group 23

In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

#### Loading the data

In [18]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


#### Network class

In [19]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

#### Training the network with SGD

In [20]:
# Logging for tensorboard
writer = SummaryWriter()

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train (SGD)", loss, epoch*len(trainloader)+i)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')
model_dict = model.state_dict()
writer.flush()
writer.close()

[1,  2000] loss: 2.309
[1,  4000] loss: 2.309
[1,  6000] loss: 2.306
[1,  8000] loss: 2.306
[1, 10000] loss: 2.307
[1, 12000] loss: 2.305
[2,  2000] loss: 2.306
[2,  4000] loss: 2.303
[2,  6000] loss: 2.304
[2,  8000] loss: 2.303
[2, 10000] loss: 2.303
[2, 12000] loss: 2.303
[3,  2000] loss: 2.302
[3,  4000] loss: 2.303
[3,  6000] loss: 2.302
[3,  8000] loss: 2.301
[3, 10000] loss: 2.301
[3, 12000] loss: 2.302
[4,  2000] loss: 2.301
[4,  4000] loss: 2.300
[4,  6000] loss: 2.299
[4,  8000] loss: 2.299
[4, 10000] loss: 2.299
[4, 12000] loss: 2.297
[5,  2000] loss: 2.297
[5,  4000] loss: 2.297
[5,  6000] loss: 2.296
[5,  8000] loss: 2.296
[5, 10000] loss: 2.295
[5, 12000] loss: 2.295
[6,  2000] loss: 2.294
[6,  4000] loss: 2.293
[6,  6000] loss: 2.292
[6,  8000] loss: 2.291
[6, 10000] loss: 2.289
[6, 12000] loss: 2.288
[7,  2000] loss: 2.287
[7,  4000] loss: 2.284
[7,  6000] loss: 2.283
[7,  8000] loss: 2.281
[7, 10000] loss: 2.279
[7, 12000] loss: 2.275
[8,  2000] loss: 2.271
[8,  4000] 

#### Testing the network with SGD

In [21]:
dataiter = iter(testloader)
images, labels = next(dataiter)

model2 = Net()
model2.load_state_dict(model_dict)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
# Learning off
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model2(images)
        _, preds = torch.max(outputs.data, 1)
        for label, pred in zip(labels, preds):
            if label == pred:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

accs = []
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    accs.append(accuracy)
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
avg_acc = sum(accs) / len(accs)
print(f'Total accuracy: {avg_acc:.1f} %')

Accuracy for class: plane is 21.2 %
Accuracy for class: car   is 10.2 %
Accuracy for class: bird  is 0.1 %
Accuracy for class: cat   is 17.4 %
Accuracy for class: deer  is 0.0 %
Accuracy for class: dog   is 8.3 %
Accuracy for class: frog  is 69.6 %
Accuracy for class: horse is 11.8 %
Accuracy for class: ship  is 54.3 %
Accuracy for class: truck is 43.4 %
Total accuracy: 23.6 %


Using SGD as the optimizer and LeakyReLU as the activation function, the model achieved 23.6% testing accuracy.

#### Training the network with Adam

In [22]:
# Logging for tensorboard
writer = SummaryWriter()

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train (Adam)", loss, epoch*len(trainloader)+i)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')
model_dict = model.state_dict()
writer.flush()
writer.close()

[1,  2000] loss: 2.032
[1,  4000] loss: 1.788
[1,  6000] loss: 1.723
[1,  8000] loss: 1.658
[1, 10000] loss: 1.652
[1, 12000] loss: 1.605
[2,  2000] loss: 1.545
[2,  4000] loss: 1.523
[2,  6000] loss: 1.521
[2,  8000] loss: 1.461
[2, 10000] loss: 1.465
[2, 12000] loss: 1.431
[3,  2000] loss: 1.421
[3,  4000] loss: 1.405
[3,  6000] loss: 1.371
[3,  8000] loss: 1.387
[3, 10000] loss: 1.376
[3, 12000] loss: 1.358
[4,  2000] loss: 1.322
[4,  4000] loss: 1.307
[4,  6000] loss: 1.326
[4,  8000] loss: 1.307
[4, 10000] loss: 1.286
[4, 12000] loss: 1.307
[5,  2000] loss: 1.272
[5,  4000] loss: 1.272
[5,  6000] loss: 1.241
[5,  8000] loss: 1.264
[5, 10000] loss: 1.225
[5, 12000] loss: 1.205
[6,  2000] loss: 1.185
[6,  4000] loss: 1.195
[6,  6000] loss: 1.190
[6,  8000] loss: 1.210
[6, 10000] loss: 1.204
[6, 12000] loss: 1.185
[7,  2000] loss: 1.132
[7,  4000] loss: 1.163
[7,  6000] loss: 1.133
[7,  8000] loss: 1.148
[7, 10000] loss: 1.157
[7, 12000] loss: 1.140
[8,  2000] loss: 1.105
[8,  4000] 

In [23]:
#### Testing the network with Adam

In [24]:
dataiter = iter(testloader)
images, labels = next(dataiter)

model2 = Net()
model2.load_state_dict(model_dict)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
# Learning off
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model2(images)
        _, preds = torch.max(outputs.data, 1)
        for label, pred in zip(labels, preds):
            if label == pred:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

accs = []
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    accs.append(accuracy)
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
avg_acc = sum(accs) / len(accs)
print(f'Total accuracy: {avg_acc:.1f} %')

Accuracy for class: plane is 69.3 %
Accuracy for class: car   is 67.1 %
Accuracy for class: bird  is 35.3 %
Accuracy for class: cat   is 41.0 %
Accuracy for class: deer  is 64.1 %
Accuracy for class: dog   is 62.1 %
Accuracy for class: frog  is 68.5 %
Accuracy for class: horse is 59.9 %
Accuracy for class: ship  is 70.7 %
Accuracy for class: truck is 70.8 %
Total accuracy: 60.9 %


Using Adam as the optimizer and LeakyReLU as the activation function, the model achieved 60.9% testing accuracy.